In [1]:
"""
《设计模式：可复用面向对象软件的基础》中提出了23种设计模式，但对于Python来说，函数是一等对象，
我们需要重新审视“策略”，“命令”，“模版方法“，和“访问者”模式
"""

'\n《设计模式：可复用面向对象软件的基础》中提出了23种设计模式，但对于Python来说，函数是一等对象，\n我们需要重新审视“策略”，“命令”，“模版方法“，和“访问者”模式\n'

In [2]:
"""
策略模式是处理算法不同变体的一种成熟模式，策略模式通过接口或抽象类封装算法的标识，
即在接口中定义一个抽象方法，实现该接口的类将实现接口中的抽象方法。策略模式把针对
一个算法标识的一系列具体算法分别封装在不同的类中，使得各个类给出的具体算法可以相互替换。

在策略模式中，封装算法标识的接口称作策略，实现该接口的类称作具体策略。
"""

'\n策略模式是处理算法不同变体的一种成熟模式，策略模式通过接口或抽象类封装算法的标识，\n即在接口中定义一个抽象方法，实现该接口的类将实现接口中的抽象方法。策略模式把针对\n一个算法标识的一系列具体算法分别封装在不同的类中，使得各个类给出的具体算法可以相互替换。\n\n在策略模式中，封装算法标识的接口称作策略，实现该接口的类称作具体策略。\n'

In [4]:
"""
我们通过下面的例子来看一下具体的策略模式的实现。
假如一个网店制定了下述具体折扣规则：
1、有1000及以上积分的顾客，每个订单享5%折扣
2、同一订单中，单个商品数量达到20个以上，享10%折扣
3、订单中的不同商品达到10个以上，享7%折扣

一个策略模式需要三个要素：上下文、策略、具体策略。
本例中的上下文即Order，用户的订单，策略是对用户订单的折扣策略，具体策略即我们上面提出的三种方案
"""
from abc import ABC,abstractmethod
from collections import namedtuple
Customer = namedtuple('Customer','name fidelity')
class LineItem:
    def __init__(self,product,quantity,price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity

class Order(): # 上下文
    def __init__(self,customer,cart,promotion = None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self,'__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total:{:.2f} due : {:.2f}>'
        return fmt.format(self.total(),self.due())
    
class Promotion(ABC):
    
    @abstractmethod
    def discount(self,order):
        pass
    
class FidelityPromo(Promotion):
    def discount(self,order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
    
class BulkItemPromo(Promotion):
    def discount(self,order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount
    
class LargeOrderPromo(Promotion):
    def discount(self,order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

In [6]:
joe = Customer('John Doe',0)
ann = Customer('Ann Smith',1100)

cart = [LineItem('banana',4,.5),
        LineItem('apple',10,1.5),
        LineItem('watermellon',5,5.0)]

print(Order(joe,cart,FidelityPromo()))
print(Order(ann,cart,FidelityPromo()))
banana_cart = [LineItem('banana',30,.5),
              LineItem('apple',10,1.5)]
print(Order(joe,banana_cart,BulkItemPromo()))

long_order = [LineItem(str(item_code),1,1.0) for item_code in range(10)]
print(Order(joe,long_order,LargeOrderPromo()))

<Order total:42.00 due : 42.00>
<Order total:42.00 due : 39.90>
<Order total:30.00 due : 28.50>
<Order total:10.00 due : 9.30>


In [11]:
"""
在python中，我们完全可以使用函数来实现策略模式。
"""
Customer = namedtuple('Customer','name fidelity')
class LineItem:
    def __init__(self,product,quantity,price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity

class Order(): # 上下文
    def __init__(self,customer,cart,promotion = None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self,'__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total:{:.2f} due : {:.2f}>'
        return fmt.format(self.total(),self.due())
    
def fidelity_promo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
    
def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount
    
def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [12]:
joe = Customer('John Doe',0)
ann = Customer('Ann Smith',1100)

cart = [LineItem('banana',4,.5),
        LineItem('apple',10,1.5),
        LineItem('watermellon',5,5.0)]

print(Order(joe,cart,fidelity_promo))
print(Order(ann,cart,fidelity_promo))
banana_cart = [LineItem('banana',30,.5),
              LineItem('apple',10,1.5)]
print(Order(joe,banana_cart,bulk_item_promo))

long_order = [LineItem(str(item_code),1,1.0) for item_code in range(10)]
print(Order(joe,long_order,large_order_promo))

<Order total:42.00 due : 42.00>
<Order total:42.00 due : 39.90>
<Order total:30.00 due : 28.50>
<Order total:10.00 due : 9.30>


In [13]:
"""
如果我们想要通过一种简单的方式，来得到最佳的策略，我们可能会进行下面的设计
"""
promos = [fidelity_promo,bulk_item_promo,large_order_promo]

def best_promo(order):
    return max(promo(order) for promo in promos)

print(Order(joe,long_order,best_promo))

<Order total:10.00 due : 9.30>


In [14]:
"""
当我们想要添加新的策略时，不得不往promos里添加新的函数名，这样会比较麻烦，那么有没有办法不用修改promos呢？这里有两种解决方案
"""


'\n当我们想要添加新的策略时，不得不往promos里添加新的函数名，这样会比较麻烦，那么有没有办法不用修改promos呢？这里有两种解决方案\n'

In [16]:
"""
1、使用python的内置函数globals(),它返回一个字典，表示当前的全局符号表，这个符号表始终针对当前模块(对函数或者方法来说，指定义她们的模块)

因此，我们可以使用globals函数找到除best_promo外其他可用的*_promo函数。
"""

promos = [globals()[name] for name in globals() if name.endswith('_promo') and name!= 'best_promo']
print(promos)
print(Order(joe,long_order,best_promo))

[<function large_order_promo at 0x106cd6950>, <function bulk_item_promo at 0x106cd6730>, <function fidelity_promo at 0x106cd6840>]
<Order total:10.00 due : 9.30>


In [17]:
"""
2、使用一个单独的模块保存所有的策略函数，并使用inspect模块进行导入
inspect.getmembers函数用于获取对象的属性，inspect.isfunction判断对象是否是一个函数
"""
import promotions
import inspect
promos = [func for name,func in inspect.getmembers(promotions,inspect.isfunction)]
print(promos)
print(Order(joe,long_order,best_promo))

[<function bulk_item_promo at 0x106cd6048>, <function fidelity_promo at 0x106baab70>, <function large_order_promo at 0x106cd6598>]
<Order total:10.00 due : 9.30>
